# Sélection des variables
## Import des données
L'étude se fera à partir de janvier 1990.

In [58]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler

# Import de la variable cible : conso

conso = pd.read_csv('Ressources/Consommation des ménages trimestrielle.csv',sep=';')
conso = conso.set_index('Période').replace({',':'.'},regex=True).astype('float64')

# Import des variables explicatives : goods, services, retail_trade

goods = pd.read_csv('Ressources/Goods index.csv',sep=';').drop(range(6))
goods = goods.rename({'Titre :':'Date'},axis='columns').set_index('Date').replace({',':'.'},regex=True).astype('float64')

services = pd.read_csv('Ressources/Services index.csv',sep=';').drop(range(4))
services = services.rename({'Titre :':'Date'},axis='columns').set_index('Date').replace({',':'.'},regex=True).astype('float64')

retail_trade = pd.read_csv('Ressources/Retail trade index.csv',sep=';').drop(range(4)).reset_index().drop('index',axis=1)
retail_trade = retail_trade.rename({'Titre :':'Date'},axis='columns').set_index('Date').replace({',':'.'},regex=True).astype('float64')

# Constitution de sous-datasets

goods_brut = goods.loc[:,[column.endswith('(Brut)') for column in goods.columns]]
goods_CVS = goods.loc[:,[column.endswith('(CVS)') for column in goods.columns]]
goods_tendance = goods.loc[:,[column.endswith('(Tendance)') for column in goods.columns]] # Pour goods

services_brut = services.loc[:,[column.endswith('(Brut)') for column in services.columns]]
services_CVS = services.loc[:,[column.endswith('(CVS)') for column in services.columns]]
services_tendance = services.loc[:,[column.endswith('(Tendance)') for column in services.columns]] # Pour services

retail_brut = retail_trade.loc[:,[column.endswith('(Brut)') for column in retail_trade.columns]]
retail_CJO_CVS = retail_trade.loc[:,[column.endswith('(CJO CVS)') for column in retail_trade.columns]]
retail_brut_valeur = retail_brut.filter(regex='indice en valeur')
retail_brut_volume = retail_brut.filter(regex='indice en volume')
retail_CJO_CVS_valeur = retail_CJO_CVS.filter(regex='indice en valeur')
retail_CJO_CVS_volume = retail_CJO_CVS.filter(regex='indice en volume') # Pour retail_trade

# Réunion dans des matrices X, X-brut, X-CVS, X-tendance

services_90 = services.loc['Déc 2019':'Jan 1990'].reset_index()
goods_90 = goods.loc[:'01/01/1990'].reset_index()
goods_90['Date'] = retail_trade.reset_index()['Date'] # On commence par ajuster les nombres de ligne et les index.

X = retail_trade.merge(services_90,on='Date').merge(goods_90,on='Date').set_index('Date')

# Constitution de la variable cible yty-var

var = []
for i in range(4,len(conso),1):
    var.append(float(conso['TOTAL '].iloc[i]/conso['TOTAL '].iloc[i-4] - 1))
yty_var = pd.DataFrame({'Variations de la consommation':var,
'Date':list(conso.index)[4:]})
yty_var = yty_var.set_index('Date')

# Normalisation des matrices

Y_norm = StandardScaler().fit_transform(yty_var)
X_norm = StandardScaler().fit_transform(X)

In [62]:
X

,"Vente à distance, indice en valeur (Brut)","Vente à distance, indice en volume (Brut)","Commerce de détail, indice en valeur (Brut)","Commerce de détail, indice en volume (Brut)","Grande distribution, indice en valeur (Brut)","Grande distribution, indice en volume (Brut)","Produits alimentaires, indice en valeur (Brut)","Produits alimentaires, indice en volume (Brut)","Produits Industriels, indice en valeur (Brut)","Produits Industriels, indice en volume (Brut)",...,"Industrie manufacturière, évolution des marges par rapport au trimestre précédent (Brut)","Industrie manufacturière, évolution du résultat brut d'exploitation par rapport au trimestre précédent (Brut)","Industrie manufacturière, situation de la trésorerie en fin de trimestre (Brut)","Industrie manufacturière, évolution des dépenses d'investissement par rapport au trimestre précédent (CVS)","Industrie manufacturière, prévisions des dépenses d'investissement à 3 mois (CVS)",Indicateur trimestriel du climat des affaires dans l'industrie manufacturière (cvs),"Industrie manufacturière, évolution des marges à l'exportation par rapport au trimestre précédent (CVS)","Industrie manufacturière, évolution des marges par rapport au trimestre précédent (CVS)","Industrie manufacturière, évolution du résultat brut d'exploitation par rapport au trimestre précédent (CVS)","Industrie manufacturière, situation de la trésorerie en fin de trimestre (CVS)"
Date,,,,,,,,,,,,,,,,,,,,,
Déc 2019,168.22,163.74,129.97,130.99,130.99,127.61,127.82,113.51,132.06,143.79,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Nov 2019,189.24,185.16,104.88,105.68,111.06,108.72,105.94,94.89,103.86,113.59,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Oct 2019,149.09,145.80,100.17,99.42,110.46,108.16,107.55,97.32,93.02,100.95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sep 2019,133.01,130.07,88.98,88.58,99.08,96.99,95.30,86.00,82.90,90.46,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aoû 2019,125.90,125.96,93.88,93.87,104.63,104.52,105.54,97.94,82.70,90.97,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Mai 1990,106.26,123.00,NaN,NaN,68.40,79.27,73.89,98.19,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Avr 1990,91.85,106.49,NaN,NaN,67.27,78.01,70.77,94.31,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Mar 1990,106.43,124.03,NaN,NaN,75.41,87.98,77.35,103.11,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Régression Lasso

In [60]:
# Le terme alpha est en fait le lambda

alpha = 1

clf = Lasso(alpha=alpha)
clf.fit(X_norm, Y_norm)
clf.coef_

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').